In [1]:
import spacy

In [2]:
def get_nps(doc):
    nchunks = []
    nheads = []
    for token in doc:
        if token.dep_ in ['nsubj', 'obj', 'iobj']:
            nheads.append(token)
    
    for token in nheads:
        nchunk = [(token)]
        for word in token.subtree:
             if word.pos_ in ['DET', 'NOUN']:
                nchunk.append(word)
        nchunks.append(nchunk)
    return nchunks

def get_nmodifiers(head, doc):
    pass

In [5]:
s = "O menino foi bem na prova difícil e a menina colou a resposta errada."
nlp = spacy.load('pt_core_news_sm')
doc = nlp(s)
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head)
nps = get_nps(doc)
print(nps)

O DET <artd>|ART|M|S|@>N det menino
menino NOUN <np-def>|N|M|S|@SUBJ> nsubj prova
foi VERB <mv>|V|PS|3S|IND|@FS-STA cop prova
bem ADV <quant>|ADV|@>A advmod na
na DET <arti>|ART|F|S|@>N case prova
prova NOUN <np-idf>|N|F|S|@<SC ROOT prova
difícil ADJ ADJ|F|S|@N< amod prova
e CCONJ <co-acc>|KC|@CO cc colou
a DET <artd>|ART|F|S|@>N det menina
menina NOUN <np-def>|N|F|S|@SUBJ> nsubj colou
colou VERB <mv>|V|PS|3S|IND|@FS-STA conj prova
a DET <artd>|ART|F|S|@>N det resposta
resposta NOUN <np-def>|N|F|S|@<ACC obj colou
errada VERB <mv>|V|PCP|F|S|@ICL-N< acl resposta
. PUNCT PU|@PU punct prova
[[menino, O, menino], [menina, a, menina], [resposta, a, resposta]]


In [7]:
s = "Os meninos foram bem na prova e as meninas."
nlp = spacy.load('pt_core_news_sm')
doc = nlp(s)
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head)
nps = get_nps(doc)
print(nps)

Os DET <artd>|ART|M|P|@>N det meninos
meninos SYM <np-def>|N|M|P|@SUBJ> nsubj foram
foram VERB <mv>|V|PS|3P|IND|@FS-STA cop prova
bem ADV <quant>|ADV|@<ADVL advmod na
na DET <arti>|ART|F|S|@>N xcomp foram
prova NOUN <np-idf>|N|F|S|@<SC ROOT prova
e CCONJ <co-acc>|KC|@CO cc meninas
as DET <artd>|ART|F|P|@>N det meninas
meninas NOUN <cjt>|<np-def>|N|F|P|@P< conj prova
. PUNCT PU|@PU punct prova
[[meninos, Os]]


# Utilizar o córpus obtido para P1
- Obter os núcleos (head) dos sintagmas nominais (SN)
- Obter os modificadores do núcleo do SN
- Para os núcleos dos sintagmas nominais, e os seus modificadores
- Criar um dicionário:
``` 
{"núcleo": [("modificador1", freq), ("modificador2", freq)]}
```
- Produzir frases, com o seguinte padrão:
```
"núcleo" verbo_ser "modificador" mais frequente.
```
- o verbo ser, deve ser conjugado como "é" para núcleos de sintagmas nominais no singular e como "são" para núcleos de sintagmas nominais no plural